<a href="https://colab.research.google.com/github/KimDongHyun0907/LG_Aimers_Hackathon/blob/main/Smart_factory_product_quality_status_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import random
import os
import numpy as np

from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
seed_everything(37) # Seed 고정

from google.colab import drive
drive.mount('/content/drive')

!unzip -qq "/content/drive/MyDrive/open.zip"

Mounted at /content/drive


In [ ]:
train_df = pd.read_csv('./train.csv')
test_df = pd.read_csv('./test.csv')

train_x = train_df.drop(columns=['PRODUCT_ID', 'TIMESTAMP', 'Y_Class', 'Y_Quality'])
train_y = train_df['Y_Class']

test_x = test_df.drop(columns=['PRODUCT_ID', 'TIMESTAMP'])

train_x = train_x.fillna(0)
test_x = test_x.fillna(0)

In [ ]:
# qualitative to quantitative
qual_col = ['LINE', 'PRODUCT_CODE']

for i in qual_col:
    le = LabelEncoder()
    le = le.fit(train_x[i])
    train_x[i] = le.transform(train_x[i])

    for label in np.unique(test_x[i]):
        if label not in le.classes_:
            le.classes_ = np.append(le.classes_, label)
    test_x[i] = le.transform(test_x[i])



In [ ]:
import lightgbm as lgb

param = {'num_leaves':10, 'objective':'binary','metric':'binary_logloss',
         'boosting_type':'dart','learning_rate':0.01}

train_data=lgb.Dataset(train_x,label=train_y.values)
#num_round=100
num_round=1000
bst=lgb.train(param,train_data, num_round)
preds=bst.predict(test_x)

preds

In [ ]:
from lightgbm import LGBMClassifier
lgbm_wrapper=LGBMClassifier(n_estimators=2000,learning_rate=0.01,boosting_type='dart',num_leaves=100,max_depth=7)
lgbm_wrapper.fit(train_x,train_y,eval_metric='logloss', verbose=True)
preds=lgbm_wrapper.predict(test_x)

In [ ]:
preds

In [ ]:
preds_sort=sorted(preds)

In [ ]:
preds_sort

In [ ]:
preds_sort[300:]

In [ ]:
preds_sort2=[]
for i in range(len(preds_sort)-1):
  preds_sort2.append((preds_sort[i+1]-preds_sort[i],preds_sort[i+1],preds_sort[i]))

preds_sort2.sort()

In [ ]:
preds_sort2

In [ ]:
def labeling(x):
  if x<0.73: return 0
  elif x>0.803: return 2
  else: return 1

preds1=pd.Series(preds).apply(labeling)
preds1

In [ ]:
submit = pd.read_csv('./sample_submission.csv')
submit['Y_Class'] = preds
submit.to_csv('./baseline_submission.csv', index=False)
submit

0.7 미만 0

0.8 초과 2

0.6011904762

====================

0.6 미만 0

0.8 초과 2

0.5699308986

====================

0.73 미만 0

0.8 초과 2

0.6103684211

====================

0.74 미만 0

0.8 초과 2

0.60011598

====================

0.73 미만 0

0.791 초과 2

0.5705877943

====================

0.73 미만 0

0.803 초과 2

0.6170347128

====================

0.738 미만 0

0.803 초과 2

0.6068054883

====================

====================

0.8 미만 0

0.967 초과 2

0.5532377227

====================

0.88 미만 0

0.967 초과 2

0.5235279626

====================

0.82 미만 0

0.967 초과 2

0.5532377227

====================

0.83 미만 0

0.967 초과 2

0.5383367486

====================